In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import seaborn           as sns
import matplotlib.dates  as mdates
import statsmodels.api   as sm
import heapq
import os


from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import DeterministicProcess

In [ ]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [ ]:
# annotations: https://stackoverflow.com/a/49238256/5769929
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv",parse_dates=["time"],index_col='row_id')
test  = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv",parse_dates=["time"],index_col='row_id')

In [ ]:
train_pivoted = train.pivot(index='time',columns=['x','y','direction'], values='congestion')
train_pivoted.columns = train_pivoted.columns = [' '.join(str(col)).strip() for col in train_pivoted.columns.values]

In [ ]:
def plot_periodogram(ts, detrend='linear', ax=None, fsm=None):
    from scipy.signal import periodogram
    if (fsm is None):
        fs =  pd.Timedelta("15 D") / pd.Timedelta("20 minutes")
    else:
        fs = fsm
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 8, 15, 30, 60, 120, 360])
    if (fsm is None):
        ax.set_xticklabels(
        [
            "Biweekly (1)",
            "Weekly   (2)",
            "Biweekly (4)",
            "Bidaily  (8)",
            "Daily   (15)",
            "HalfDay (30)",
           "Sixhour (60)",
           "Threehour (120)",
           "Hourly  (360)"
        ],
        rotation=30,
        )
    else:        
        ax.set_xticklabels(
        [
            "(1)",
            "(2)",
            "(4)",
            "(8)",
            "(15)",
            "(30)",
           "(60)",
           "(120)",
           "(360)"
        ],
        rotation=30,
        )
        
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [ ]:
print(train_pivoted.head())

In [ ]:
#for col in train_pivoted.columns:
#    print(col)
#    print(train_pivoted[col].value_counts())
#    print('##################################')

In [ ]:
#train = downcast_dtypes(train)
#test  = downcast_dtypes(test)

In [ ]:
train_pivoted.describe().T

In [ ]:
print("Training dataset column name, Training dataset column type")
for col in train_pivoted.columns:
    print(col,train_pivoted[col].dtype)

In [ ]:
missing_data_count = train_pivoted.isnull().sum()
print("Missing Data in Training Set:")
print(missing_data_count)
sns.heatmap(train_pivoted.isnull())
plt.title("Missing Data in Training Set")
plt.show()

total_data = np.product(train_pivoted.shape)
total_missing = missing_data_count.sum()
percent_missing = (total_missing/total_data)*100
print("\nPercentage of Missing Data:")
print(percent_missing)

In [ ]:
train_pivoted.columns

In [ ]:
fig, _ = plt.subplots(17,4)
fig.suptitle("Congestion for every 20 minutes",fontsize=16)
train_rolled = train_pivoted.rolling(3*1).mean()
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        ax.plot(train_pivoted.index,train_pivoted[col],color='blue')
        ax.plot(train_rolled.index,train_rolled[col],color='red')
        ax.set_title(f'x, y, dir ={col}')
        locator = mdates.MonthLocator()
        ax.xaxis.set_major_locator(locator)
        plt.xticks(rotation = 90)
        if (i == 1):
            plt.legend()
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
fig.suptitle("Congestion averaged for every 6 hours",fontsize=16)
train_rolled = train_pivoted.rolling(3*6).mean()
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        ax.plot(train_pivoted.index,train_pivoted[col],color='blue')
        ax.plot(train_rolled.index,train_rolled[col],color='red')
        ax.set_title(f'x, y, dir ={col}')
        locator = mdates.MonthLocator()
        ax.xaxis.set_major_locator(locator)
        plt.xticks(rotation = 90)
        if (i == 1):
            plt.legend()
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
fig.suptitle("Congestion averaged for every 24 hours",fontsize=16)
train_rolled = train_pivoted.rolling(3*24).mean()
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        ax.plot(train_pivoted.index,train_pivoted[col],color='blue')
        ax.plot(train_rolled.index,train_rolled[col],color='red')
        ax.set_title(f'x, y, dir ={col}')
        locator = mdates.MonthLocator()
        ax.xaxis.set_major_locator(locator)
        plt.xticks(rotation = 90)
        if (i == 1):
            plt.legend()
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
fig.suptitle("Congestion averaged for every 7 days",fontsize=16)
train_rolled = train_pivoted.rolling(3*24*7).mean()
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        ax.plot(train_pivoted.index,train_pivoted[col],color='blue')
        ax.plot(train_rolled.index,train_rolled[col],color='red')
        ax.set_title(f'x, y, dir ={col}')
        locator = mdates.MonthLocator()
        ax.xaxis.set_major_locator(locator)
        plt.xticks(rotation = 90)
        if (i == 1):
            plt.legend()
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
train_rolled = train_pivoted.rolling(3*24*7).mean()
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        plot_periodogram(train_pivoted[col], detrend='linear', ax=ax)
        ax.set_title(f'x, y, dir ={col}')
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
train_rolled = train_pivoted.rolling(3*24*7).mean()
fs =  pd.Timedelta("7 D") / pd.Timedelta("20 minutes")
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        plot_periodogram(train_pivoted[col], detrend='linear', ax=ax,fsm=fs)
        ax.set_title(f'x, y, dir ={col}')
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
train_rolled = train_pivoted.rolling(3*24*7).mean()
fs =  pd.Timedelta("1 D") / pd.Timedelta("20 minutes")
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        plot_periodogram(train_pivoted[col], detrend='linear', ax=ax,fsm=fs)
        ax.set_title(f'x, y, dir ={col}')
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
train_rolled = train_pivoted.rolling(3*24*7).mean()
fs =  pd.Timedelta("12 hours") / pd.Timedelta("20 minutes")
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        plot_periodogram(train_pivoted[col], detrend='linear', ax=ax,fsm=fs)
        ax.set_xlim(0.1,15)
        ax.set_title(f'x, y, dir ={col}')
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

In 12 hours 4 series is enough.

###########################

In [ ]:
fig, _ = plt.subplots(17,4)
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        sm.graphics.tsa.plot_pacf(train_pivoted[col], lags=12, method="ywm", ax=ax,title =f'x, y, dir ={col}')
fig.set_size_inches(18,72)
fig.tight_layout()


6 previous lags are enough.

In [ ]:
fig, _ = plt.subplots(17,4)
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        sm.graphics.tsa.plot_pacf(train_pivoted.iloc[::3,:][col], lags=24, method="ywm", ax=ax,title =f'x, y, dir ={col}')
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        sm.graphics.tsa.plot_pacf(train_pivoted.iloc[::3*3,:][col], lags=24, method="ywm", ax=ax,title =f'x, y, dir ={col}')
fig.set_size_inches(18,72)
fig.tight_layout()

In [ ]:
fig, _ = plt.subplots(17,4)
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        sm.graphics.tsa.plot_pacf(train_pivoted.iloc[::24*3,:][col], lags=7, method="ywm", ax=ax,title =f'x, y, dir ={col}')
fig.set_size_inches(18,72)
fig.tight_layout()

# Forcasting the Trend

In [ ]:
dp = DeterministicProcess(
    index=train_pivoted.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)
# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()

X.head()

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        X_fore = dp.out_of_sample(steps=24*3)
        y_fore = pd.Series(model.predict(X_fore), index=pd.date_range(start='1991-09-30 12:00:00', end='1991-10-01 11:40:00',freq='20 min'))
        #y_fore.head()
        
        train_pivoted[col].plot(title= col + " - Trend Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim(0.1,15)
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

# Forcasting the Seasonality

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(freq="15D", order=60)  # 

dp2 = DeterministicProcess(
    index=train_pivoted.index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    #seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp2.in_sample()  # create features for dates in tunnel.index

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        X_fore = dp2.out_of_sample(steps=24*3,forecast_index=pd.date_range(start='1991-09-30 12:00:00', end='1991-10-01 11:40:00',freq='20 min'))
        y_fore = pd.Series(model.predict(X_fore), index=pd.date_range(start='1991-09-30 12:00:00', end='1991-10-01 11:40:00',freq='20 min'))
        #y_fore.head()
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

 ***Lets have a closer look***

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        X_fore = dp2.out_of_sample(steps=24*3,forecast_index=pd.date_range(start='1991-09-30 12:00:00', end='1991-10-01 11:40:00',freq='20 min'))
        y_fore = pd.Series(model.predict(X_fore), index=pd.date_range(start='1991-09-30 12:00:00', end='1991-10-01 11:40:00',freq='20 min'))
        #y_fore.head()
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

# What about lags?
***Lets go with 6 lags for now***

In [ ]:
def make_lags(ts, lags,freq="20Min"):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i,freq)
            for i in range(1, lags + 1)
        },
        axis=1)

In [ ]:
dp = DeterministicProcess(
    index=train_pivoted.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)
# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()

X.head()

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        train_lagged = make_lags(train_pivoted[col],6)
        train_lagged = train_lagged.fillna(0.0)
        X = train_lagged
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        #y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

***It seems that this guy is more cyclic rather than seasonal!!!***

# Let's do some predictions!!!!

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        train_lagged = make_lags(train_pivoted[col],lags=8,freq="3H")
        train_lagged = train_lagged.fillna(0.0)
        X = train_lagged
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        #y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

**Hahaha!! With 8 lags having periods of 3 hours it becomes terrible**

Let's go with 24 lags and period of 1 hour

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        train_lagged = make_lags(train_pivoted[col],lags=24,freq="1H")
        train_lagged = train_lagged.fillna(0.0)
        X = train_lagged
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        #y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

**No good**

Let's go with 24 lags and period of 20 minutes

In [ ]:
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        train_lagged = make_lags(train_pivoted[col],lags=24*3,freq="20Min")
        train_lagged = train_lagged.fillna(0.0)
        X = train_lagged
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        #y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

**Lets go with XGBoost**

It might take a while!
We should wait for a moment .....

In [ ]:
from xgboost import XGBRegressor
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        train_lagged = make_lags(train_pivoted[col],lags=24*3,freq="20Min")
        train_lagged = train_lagged.fillna(0.0)
        X = train_lagged
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = XGBRegressor()
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        #y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        #ax.set_xlim("1991-09-13","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()

**It is awesome!!!**

Let's have a closer look

In [ ]:
from xgboost import XGBRegressor
fig, _ = plt.subplots(17,4)
train_size = int(0.66*X.shape[0])
for i, col in enumerate(train_pivoted.columns):
        ax = plt.subplot(17,4,i+1)
        train_lagged = make_lags(train_pivoted[col],lags=24*3,freq="20Min")
        train_lagged = train_lagged.fillna(0.0)
        X = train_lagged
        y = train_pivoted[col]
        X_train = X[:train_size]
        y_train = y[:train_size]
        X_valid = X[train_size:]
        y_valid = y[train_size:]
        model = XGBRegressor()
        model.fit(X_train,y_train)
        
        
        y_train_pred = pd.Series(model.predict(X_train), index=X_train.index)
        y_valid_pred = pd.Series(model.predict(X_valid), index=X_valid.index)
        
        train_pivoted[col].plot(title= col + " - Seasonality Forecast", ax=ax)
        y_train_pred.plot(ax=ax, linewidth=3, label="Training", color="C1")
        y_valid_pred.plot(ax=ax, linewidth=3, label="Validation", color="C2")
        #y_fore.plot(ax=ax, linewidth=3, label="Forecast", color="C3")
        _ = ax.legend()
        
        ax.set_xlim("1991-09-28","1991-10-02")
        plt.xticks(rotation = 90)
fig.set_size_inches(18,72)
fig.tight_layout()